In [1]:
import afqinsight as afqi
import copt
from sklearn.model_selection import train_test_split

%load_ext line_profiler

In [2]:
afq_data = afqi.load_afq_data('../data/raw/als_data', target_cols=['class'], binary_positives=['ALS'])

x, y, groups, columns, bias_index = (
    afq_data.x,
    afq_data.y['class'],
    afq_data.groups,
    afq_data.columns,
    afq_data.bias_index
)

/Users/richford/miniconda3/envs/afq-insight/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42
)

In [5]:
%lprun -T lprof_text0 -D lprof_dump -f afqi.sgl_estimator -f copt.utils.LogLoss.f_grad -f copt.minimize_PGD -f afqi.prox.SparseGroupL1.prox afqi.sgl_estimator(x_train=X_train, y_train=y_train, x_test=X_test, y_test=y_test, groups=groups, bias_index=bias_index, alpha1=0.01, alpha2=0.01)


*** Profile stats pickled to file 'lprof_dump'. 

*** Profile printout saved to text file 'lprof_text0'. 


Timer unit: 1e-06 s

Total time: 7.12026 s
File: /Users/richford/miniconda3/envs/afq-insight/lib/python3.6/site-packages/copt/proxgrad.py
Function: minimize_PGD at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def minimize_PGD(
     8                                                   f_grad, x0, prox=None, tol=1e-6, max_iter=500, verbose=0,
     9                                                   callback=None, backtracking=True, step_size=None,
    10                                                   max_iter_backtracking=1000, backtracking_factor=0.6,
    11                                                   ):
    12                                               """Proximal gradient descent.
    13                                           
    14                                               Solves problems of the form
    15                                           
    16                                    

In [ ]:
sg1 = afqi.prox.SparseGroupL1(0.1, 0.1, groups=groups, bias_index=bias_index)

In [ ]:
import numpy as np
n_features = X_train.shape[1]
beta0 = np.zeros(n_features)
    
fk, grad_fk = copt.utils.LogLoss(X_train, y_train).f_grad(beta0)

In [ ]:
%lprun -f afqi.prox.SparseGroupL1.prox sg1.prox(beta0 - 1 * grad_fk, 1)

In [ ]:
beta0.shape

In [ ]:
test_beta0 = np.random.rand(16001)

In [ ]:
test_beta0.shape

In [ ]:
test_beta0[groups[0]]

In [ ]:
norms = np.linalg.norm(test_beta0[groups], axis=1)

In [ ]:
out_loop = np.random.rand(len(beta0))
out_vec = np.copy(out_loop)

In [ ]:
#%%timeit
for i, g in enumerate(groups):
    if norms[i] > norms.mean():
        out_loop[g] -= 0.1 * out_loop[g] / norms[i]
    else:
        out_loop[g] = 0

In [ ]:
#%%timeit
norm_mask_bool = norms > norms.mean()
norm_mask_idx = np.where(norm_mask_bool)[0]
group_mask = np.array(groups)[norm_mask_idx]
not_group_mask = np.array(groups)[np.where(np.logical_not(norm_mask_bool))]
out_vec[group_mask] -= 0.1 * out_vec[group_mask] / norms[norm_mask_idx, np.newaxis]
out_vec[not_group_mask] = 0.0

In [ ]:
np.array(groups)[np.where(np.logical_not(norm_mask_bool))]

In [ ]:
not_group_mask

In [ ]:
np.allclose(out_loop, out_vec)

In [ ]:
groups

In [ ]:
num = np.ones((3, 5), dtype=np.float64)
denom = np.array([1, 2, 3], dtype=np.float64)
print(num.shape)
print(denom.shape)
num / denom[:, np.newaxis]

In [6]:
import hyperopt as hp

In [10]:
t = hp.Trials()
tm = hp.mongoexp.MongoTrials()

TypeError: __init__() missing 1 required positional argument: 'arg'

hyperopt.base.Trials